# reid

In [26]:
import argparse
import logging
import os
import sys
from pathlib import Path

import numpy as np

sys.path.append(".")

from centroids_reid.config import cfg
from centroids_reid.train_ctl_model import CTLModel

from centroids_reid.inference.inference_utils import (
    ImageDataset,
    ImageFolderWithPaths,
    calculate_centroids,
    create_pid_path_index,
    make_inference_data_loader,
    run_inference,
)

### Prepare logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))
log = logging.getLogger(__name__)

### Functions used to extract pair_id
exctract_func = (
    lambda x: (x).rsplit(".", 1)[0].rsplit("_", 1)[0]
)  ## To extract pid from filename. Example: /path/to/dir/product001_04.jpg -> pid = product001
exctract_func = lambda x: Path(
    x
).parent.name  ## To extract pid from parent directory of an iamge. Example: /path/to/root/001/image_04.jpg -> pid = 001


parser = argparse.ArgumentParser(
    description="Create embeddings for images that will serve as the database (gallery)"
)
parser.add_argument(
    "--config_file", default="", help="path to config file", type=str
)
parser.add_argument(
    "--images-in-subfolders",
    help="if images are stored in the subfloders use this flag. If images are directly under DATASETS.ROOT_DIR path do not use it.",
    action="store_true",
)
parser.add_argument(
    "--print_freq",
    help="number of batches the logging message is printed",
    type=int,
    default=10,
)
parser.add_argument(
    "opts",
    help="Modify config options using the command-line",
    default=None,
    nargs=argparse.REMAINDER,
)
args = parser.parse_args()
if args.config_file != "":
    cfg.merge_from_file(args.config_file)
cfg.merge_from_list(args.opts)

### Data preparation
if args.images_in_subfolders:
    dataset_type = ImageFolderWithPaths
else:
    dataset_type = ImageDataset
log.info(f"Preparing data using {dataset_type} dataset class")
val_loader = make_inference_data_loader(cfg, cfg.DATASETS.ROOT_DIR, dataset_type)
if len(val_loader) == 0:
    raise RuntimeError("Lenght of dataloader = 0")

### Build model
model = CTLModel.load_from_checkpoint(cfg.MODEL.PRETRAIN_PATH)

### Inference
log.info("Running inference")
embeddings, paths = run_inference(
    model, val_loader, cfg, print_freq=args.print_freq
)

### Create centroids
log.info("Creating centroids")
if cfg.MODEL.USE_CENTROIDS:
    pid_path_index = create_pid_path_index(paths=paths, func=exctract_func)
    embeddings, paths = calculate_centroids(embeddings, pid_path_index)

### Save
SAVE_DIR = Path(cfg.OUTPUT_DIR)
SAVE_DIR.mkdir(exist_ok=True, parents=True)

log.info(f"Saving embeddings and index to {str(SAVE_DIR)}")
np.save(SAVE_DIR / "embeddings.npy", embeddings)
np.save(SAVE_DIR / "paths.npy", paths)


usage: ipykernel_launcher.py [-h] [--config_file CONFIG_FILE]
                             [--images-in-subfolders]
                             [--print_freq PRINT_FREQ]
                             ...
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"fec70963-3c3f-4d1f-bbf0-deb9d718775f" --shell=9002 --transport="tcp" --iopub=9004 --f=/tmp/tmp-356vF0qMose1CGr.json


SystemExit: 2

/home/shivani-nandani/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!python ./centroids_reid/inference/create_embeddings.py --config_file="centroids_reid/configs/256_resnet50.yml" GPU_IDS [0] DATASETS.ROOT_DIR 'street_persons_frames/' TEST.IMS_PER_BATCH 128 OUTPUT_DIR 'output_dir' TEST.ONLY_TEST True MODEL.PRETRAIN_PATH "centroids_reid/market1501_resnet50_256_128_epoch_120.ckpt"

# bounding boxes

In [13]:
import cv2

def drawBoundingBoxes(imageData, imageOutputPath, inferenceResults, color):
    """Draw bounding boxes on an image.
    imageData: image data in numpy array format
    imageOutputPath: output image file path
    inferenceResults: inference results array off object (l,t,r,b)
    colorMap: Bounding box color candidates, list of RGB tuples.
    """
    for res in inferenceResults:
        left = int(res['left'])
        top = int(res['top'])
        right = int(res['right'])
        bottom = int(res['bottom'])
        label = res['label']
        imgHeight, imgWidth, _ = imageData.shape
        thick = int((imgHeight + imgWidth) // 900)
        # print(left, top, right, bottom)
        cv2.rectangle(imageData,(left, top), (right, bottom), color, thick)
        cv2.putText(imageData, label, (left, top - 12), 0, 1e-3 * imgHeight, color, thick)
    cv2.imwrite(imageOutputPath, imageData)


In [27]:
import pandas as pd

reid_results = pd.read_csv('./street_reid_v6.csv')
reid_results

,frame#,filename,id
0,0,frame0_2.jpg,0
1,0,frame0_3.jpg,1
2,0,frame0_6.jpg,2
3,1,frame1_17.jpg,0
4,1,frame1_19.jpg,1
...,...,...,...
1096,328,frame328_4700.jpg,13
1097,328,frame328_4701.jpg,14
1098,329,frame329_4712.jpg,13
1099,329,frame329_4713.jpg,14


In [28]:
detection_results = pd.read_csv('./street_detection_results.csv')
detection_results

,xmin,ymin,xmax,ymax,confidence,class,name,frame
0,752.256592,460.979584,911.916870,628.016052,0.864604,58,potted plant,0
1,446.977387,424.172180,533.338318,542.409424,0.849613,56,chair,0
2,248.317841,354.916840,324.692047,539.327026,0.843955,0,person,0
3,325.608185,353.390533,389.656036,539.255188,0.832076,0,person,0
4,1576.837402,223.440277,1911.922974,587.946045,0.801511,58,potted plant,0
...,...,...,...,...,...,...,...,...
4719,506.333313,193.240234,574.468018,279.676575,0.416587,58,potted plant,329
4720,763.097473,478.126556,887.178772,638.719299,0.381419,58,potted plant,329
4721,69.501938,439.298340,145.456009,517.869080,0.379135,58,potted plant,329
4722,494.676575,408.958832,559.195679,567.350708,0.366885,58,potted plant,329


In [29]:
# detection_results.xmin[int(reid_results.filename[0].split('_')[1].split('.')[0])]
reid_results.filename[0].split('_')[0]

'frame0'

In [30]:
for i in range(len(reid_results)):

    imgcv = cv2.imread('./street_reid_frames/'+reid_results.filename[i].split('_')[0]+'.jpg')
    color = (255,0,0)
    results = [
        {
            'left' : detection_results.xmin[int(reid_results.filename[i].split('_')[1].split('.')[0])],
            'top' : detection_results.ymin[int(reid_results.filename[i].split('_')[1].split('.')[0])],
            'right' : detection_results.xmax[int(reid_results.filename[i].split('_')[1].split('.')[0])],
            'bottom' : detection_results.ymax[int(reid_results.filename[i].split('_')[1].split('.')[0])],
            'label': 'P'+str(reid_results.id[i])
        }
    ]
    drawBoundingBoxes(imgcv, './street_reid_frames/'+reid_results.filename[i].split('_')[0]+'.jpg', results, color)